# Build image

Here is a look at the `docker build` command, its features and options.

## Build path

The `build` command requires one argument: the path to the build folder. This argument specifies which folder will be used as the working directory for the build process. All relative paths will be considered relative to this specified path.

---

As an example, consider the configuration created in the following cell.

In [41]:
mkdir build_path
cat << EOF > build_path/dockerfile
FROM alpine
COPY message message
CMD ["cat", "message"]
EOF

echo "I'm external message" > build_path/message

mkdir build_path/folder
echo "I'm nested message" > build_path/folder/message

Here is a Dockerfile that modifies the `alpine` image to print the `message` file from the build folder. There are two `message` files located in different folders.

For better understanding, here is the file tree:

In [42]:
tree build_path

build_path
├── dockerfile
├── folder
│   └── message
└── message

1 directory, 3 files


Here is an image that uses `build_path` as the build folder. Container using this image prints `I'm external message`.

In [43]:
docker build \
    -t external_image \
    -f ./build_path/dockerfile \
    ./build_path &> /dev/null
docker run --rm external_image

I'm external message


**Note** In the examples, the `-f` option is used to specify the path to the docker file, which is generally not the same as the building path.

Exactly the same folder, but using a different build folder - prints a different message. 

In [44]:
docker build \
    -t nested_image \
    -f ./build_path/dockerfile \
    ./build_path/folder &> /dev/null
docker run --rm nested_image

I'm nested message


Getting rid of files and images we created for examples.

In [45]:
rm -r build_path
docker rmi external_image nested_image

Untagged: external_image:latest
Deleted: sha256:4d888a7322bda81b44c934cf6079a5637086150f86490c59862d3b6c1fa2804f
Untagged: nested_image:latest


## Image name (-t)

You can set the image name (tag) using the `-t <image name>` option.

---

The following cell shows the process of creating a docker image named `super_duper_puper`.

In [8]:
mkdir image_name
cat << EOF > image_name/dockerfile
FROM alpine
EOF

docker build -t super_duper_puper ./image_name &> /dev/null
docker images | grep super_duper_puper

docker rmi super_duper_puper &> /dev/null
rm -r image_name

super_duper_puper                                                   latest           64a95b915f9e   3 weeks ago     7.8MB


## Select dockerfile (`-f`)

You can pass the filepath to the docker image in the `-f` option. If your file specifying the build directive has a name different from `dockerfile`, you will need to specify the actual name as the `-f` parameter.

---
Following cell defiens docker file with name `python_apline`

In [22]:
mkdir select_dockerfile
cat << EOF > select_dockerfile/python_alpine
FROM alpine
RUN apk add python3
EOF

If you're just trying to build it, specify the path to the folder with the dockerfile.

In [23]:
docker build ./select_dockerfile

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.0s (1/1) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2B                                         0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory


: 1

You have to specify path to the docker file in `-f` option, only in such case image will build properly.

In [24]:
docker build \
    -t temp_image \
    -f ./select_dockerfile/python_alpine . &> /dev/null
docker images | grep temp_image

temp_image                    latest        d4d220b7f3bf   20 minutes ago   50.7MB


In [ ]:
docker rmi temp_image &> /dev/null
rm -r ./select_dockerfile

## Ignore files

You can ignore some files with a `.dockerignore` file. Any files specified in this file will be ignored by docker when building the image.

---

The following cell creates:

- `dockerfile` that tries to copy `folder` and pring it's content during container start.
- `.dockerignore` that counts `folder`.
- And `folder` that contains `file`.

In [26]:
mkdir folder
echo "message" > folder/file

# .dockerignore
cat << EOF > .dockerignore
folder
EOF

# dockerfile
cat << EOF > dockerfile
FROM alpine
COPY folder folder
CMD ["ls", "folder"]
EOF

If you try to build an image from such a configuration, you'll get an error - `"/folder": not found` in the `COPY` step. This happens because `.dockerignore` has banned this file.

In [27]:
docker build -t test_image .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (6/6) FINISHED                                 docker:default
 => [internal] load build definition from dockerfile                       0.0s
 => => transferring dockerfile: 89B                                        0.0s
 => [internal] load metadata for docker.io/library/alpine:latest           0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 47B                                           0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/2] FROM docker.io/library/alpine:latest                             0.0s
 => ERROR [2/2] COPY folder folder                                         0.0s
------
 > [2/2] COPY folder folder:
------
dockerfile:2
--------------------
   1 |     FROM alpine
   2 | >>> COPY fold

: 1

Ok, let's fix `.dockerignore` to reject `folder/ignored_file`. Add such a file, and another `folder/new_file`.

In [28]:
cat << EOF > .dockerignore
folder/ignored_file
EOF
echo "message" > folder/ignored_file
echo "message" > folder/new_file

Now the image has been successfully built. And in the output of the container you can see `new_file`, but not `ignored_file`, which was rejected by `.dockerignore`.

In [29]:
docker build -t test_image . &> /dev/null
docker run --rm test_image

file
new_file


Don't forget to keep your environment clean.

In [30]:
rm -r .dockerignore dockerfile folder
docker rmi test_image

Untagged: test_image:latest
Deleted: sha256:35bbd1334f9243483ff527d8bee8594f18a50636c8d02ba39e0111c3393db3bc
